In [2]:
import jax

In [3]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [12]:
def show_first_10_lines(file_path):
    try:
        with open(file_path, 'r') as file:
            for i, line in enumerate(file):
                if i < 30:
                    print(f"{i+1}: {line.strip()}")
                else:
                    break
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except IOError:
        print(f"Error: There was an issue reading the file '{file_path}'.")

# Example usage
file_path = 'projects/charformer/data/TinyStoriesV2-GPT4-train.txt'
show_first_10_lines(file_path)

1: 
2: Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!
3: He said, “Wow, that is a really amazing vase! Can I buy it?”
4: The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”
5: So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was.
6: And that's how Ben found an amazing vase in the store!
7: <|endoftext|>
8: Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They all loved to play and swim together.
9: One day, Ollie's mom said, "Ollie, hurry and get some fish for dinner!" Ollie swam fast to cat

In [336]:
import projects.charformer.data as data
from importlib import reload
data = reload(data)
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
# %pip install tqdm


In [337]:
ds = data.CharDataset(data.CharDataset.get_default_config())

In [338]:
ds.create_packed_records(input_file_path=file_path, output_dir='projects/charformer/data/tfrecords/', custom_delimiter='<|endoftext|>')

5001it [00:00, 7557.05it/s]


ArithmeticError: 

In [339]:
retokenized = ds.load_and_retokenize_tfrecord(file_path='projects/charformer/data/tfrecords/record_121.tfrecord')

In [340]:
for r in retokenized:
    print(r)

One day, a boy named Tim was eager to sort his toy cars. He had a big box with many cars. Tim wanted to put the cars in small groups. He thought it would be fun.
Tim's mom said, "Be careful with the small cars. They may get lost." Tim said, "I will be careful, Mom." He started to sort his cars by color. He had red cars, blue cars, and green cars. Tim was happy.
But then, Tim found a tiny car. It was so small, it could fit in a cell. He put the tiny car in a safe place so it would not get lost. Later, Tim showed his mom all the sorted cars. His mom was proud of him. And the tiny car was safe, just like his mom said.
<|endoftext|
One day, a little girl named Lucy went to the park with her mom. Lucy saw a tall giraffe with a long neck. She felt nervous because she had never seen a giraffe before. She held her mom's hand tight.
Lucy asked her mom a question, "Mom, why does the giraffe have such a long neck?" Her mom smiled and said, "The giraffe has a long neck so it can eat leaves from th

In [198]:
ds.create_iterator('projects/charformer/data/tfrecords/record_*.tf_record', batch_size=4)

array([1, 2])

In [212]:
np.where(q == 1)

(array([0, 1, 2]),)

In [65]:
'“'

'“'